In [3]:
# directory handling 
import os
import glob
import shutil
import pathlib

# wir gehen davon aus das die daten ungezippt im data ordner liegen
# wenn nur entzipped wurde muss der skin.cancer ordner umgenannt werden zu data
if(os.path.exists('../skin.cancer') == True):
    os.rename('../skin.cancer', '../data')

# einen abgabe ordner erstellen um sicher zu gehen keine probleme in der abgabe zu haben
pathlib.Path('data/abgabe_dir').mkdir(parents=True, exist_ok=True)

# diese ordner werden in den einzelnen modellen verwendet als referenz

train_dir = '../data/abgabe_dir/train'
validation_dir = '../data/abgabe_dir/validation'
test_dir = '../data/abgabe_dir/test'

# da der datensatz auf train und test aufgeteilt ist, und wir train, test, validation brauchen müssen wir die daten zuerst zusammenführen
all_data_in_one_folder = '../data/abgabe_dir/all_pictures'

# train und test vom original in einem gemeinsame ordner kopieren 
if(os.path.exists(all_data_in_one_folder) == False):
    shutil.copytree('../data/train', all_data_in_one_folder, dirs_exist_ok=True)
    shutil.copytree('../data/test', all_data_in_one_folder, dirs_exist_ok=True)


# eine methode um den abgabe ordner mit inhalt zu füllen

def create_abgabe_path(label, size_of_train, size_of_validation, size_of_test):
    train_dir_label = f"../data/abgabe_dir/train/{label}"
    validation_dir_label = f"../data/abgabe_dir/validation/{label}"
    test_dir_label = f"../data/abgabe_dir/test/{label}"

    src_dir = f"../data/abgabe_dir/all_pictures/{label}"

    # wenn die ordner noch nicht existieren, erstellen wir ihn
    pathlib.Path(train_dir_label).mkdir(parents=True, exist_ok=True)
    pathlib.Path(validation_dir_label).mkdir(parents=True, exist_ok=True)
    pathlib.Path(test_dir_label).mkdir(parents=True, exist_ok=True)

    # kopieren der datei anhand der angegebenen größe in der methode
    fnames = ['{}.jpg'.format(i) for i in range(1,size_of_train)]
    for fname in fnames:
        src = src_dir + "/" + fname
        dst = train_dir_label + "/" + fname
    
        try:
            shutil.copyfile(src, dst)
        except:
            print("File not found: " + src)
            
    # kopieren der datei anhand der angegebenen größe in der methode
    # da wir nicht wollen das die gleichen bilder in train und validation sind, fangen wir bei size_of_train an
    fnames = ['{}.jpg'.format(i) for i in range(size_of_train, size_of_train+size_of_validation)]
    for fname in fnames:
        src = src_dir + "/" + fname
        dst = validation_dir_label + "/" + fname    
        
        try:
            shutil.copyfile(src, dst)
        except:
            print("File not found: " + src)
            
    # kopieren der datei anhand der angegebenen größe in der methode
    # da wir nicht wollen das die gleichen bilder in validation und test sind, fangen wir bei size_of_train + size_of_validation an
    fnames = ['{}.jpg'.format(i) for i in range(size_of_train + size_of_validation, size_of_train + size_of_validation + size_of_test)]
    for fname in fnames:
        src = src_dir + "/" + fname
        dst = test_dir_label + "/" + fname

        try:
            shutil.copyfile(src, dst)
        except:
            print("File not found: " + src)
            

# das ganze wird einmal für benign und einmal für malignant ausgeführt, somit könnte man eventuell die größe der daten anpassen
create_abgabe_path("benign", size_of_train=1000, size_of_validation=250, size_of_test=250)
create_abgabe_path("malignant", size_of_train=1000, size_of_validation=250, size_of_test=250)






File not found: ../data/abgabe_dir/all_pictures/malignant/767.jpg
File not found: ../data/abgabe_dir/all_pictures/malignant/776.jpg
File not found: ../data/abgabe_dir/all_pictures/malignant/788.jpg


In [4]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16


# eine methode um die modelle einfacher und nachvollziehbarer zu erstellen

def build_basic_model(dropout=0):


    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                            input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    # dropout 
    if dropout != 0:
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))


    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-4),
                  metrics=['acc'])
    return model




In [8]:


from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


model = build_basic_model()

# All images will be [0,1] standardized
train_datagen = ImageDataGenerator(rescale=1.0/255)


k_fold_dir = "../data/abgabe_dir/all_pictures/"

# zwischenspeichern der daten für die k-fold validation denn man muss die bilder dynamisch aufteilen und das geht mit flow_from_directory nicht

training_generator = train_datagen.flow_from_directory(
    k_fold_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='binary',
    subset='training')

# die liste die wir brauchen um die daten zu speichern
training_data = []

# da flow_from_directory nur die bilder in batches zurückgibt, müssen wir die anzahl der bilder in einem ordner kennen um zu wissen wann wir aufhören müssen
max_value = (len(os.listdir(k_fold_dir+"/benign")) + len(os.listdir(k_fold_dir+"/malignant")))
print("the number of pictures in this dic is:",max_value)
for index, x in enumerate(training_generator):
    
    # die label info ist in einer liste mit einem element, deswegen müssen wir das element aus der liste holen
    if(1 in x[1] ):
        temp_to_add = 1
    else:
        temp_to_add = 0

    # da die batch_size 1 ist, ist die liste mit den bildern auch nur ein element lang
    # diese liste wird hier übersprungen damit eine klare liste mit den bildern und den labels entsteht
    for i in x[0]:
        training_data.append([i, temp_to_add])

    # abbruch wenn alle bilder durchlaufen wurden
    if(index % max_value == 0 and index != 0):
        break
    # nur um zu sehen wie weit die schleife vorangeschritten ist
    if(index % 100 == 0):
        print(index)

Found 3297 images belonging to 2 classes.
the number of pictures in this dic is: 3297
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


In [18]:
from sklearn.model_selection import KFold
import numpy as np

# Define the number of folds
k = 3


kf = KFold(n_splits=k)
scores = []



# Über die folds iterieren
for train_index, test_index in kf.split(training_data):
    # Get the training and test sets for this fold
    
    # die daten müssen numpys sein alles andere synergiert nicht mit kf.split
    temp_train = np.array(training_data)[train_index]
    temp_validate = np.array(training_data)[test_index]
    
    # die labels und die bilder müssen getrennt werden damit sie ordentlich in die model.fit funktion passen
    labels = np.array([x[1] for x in temp_train])
    images = np.array([x[0] for x in temp_train])

    # die labels und die bilder müssen getrennt werden damit sie ordentlich in die model.evaluate funktion passen
    labels_validate = np.array([x[1] for x in temp_validate])
    images_validate = np.array([x[0] for x in temp_validate])

    model.fit(images ,labels , epochs=2, verbose=1, batch_size=20)

    # Verbindung mit den validierungsdaten und ausgabe des scores für diesen fold
    x, score = model.evaluate(images_validate, labels_validate, verbose=0, steps=25)

    # Speichern des scores für diesen fold
    scores.append(score)
    print(score)

# berechnen des durchschnittlichen scores
mean_score = np.mean(scores)

C:\Users\Bene\AppData\Local\Temp/ipykernel_20924/3089571554.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_train = np.array(training_data)[train_index]
C:\Users\Bene\AppData\Local\Temp/ipykernel_20924/3089571554.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_validate = np.array(training_data)[test_index]


2968 2968
446767104 2968
Epoch 1/2
149/149 [==============================] - 218s 1s/step - loss: 0.6467 - acc: 0.6354
Epoch 2/2
149/149 [==============================] - 219s 1s/step - loss: 0.5180 - acc: 0.7429


0.7636363506317139
2968 2968
446767104 2968
Epoch 1/2
149/149 [==============================] - 210s 1s/step - loss: 0.4766 - acc: 0.7658
Epoch 2/2
149/149 [==============================] - 212s 1s/step - loss: 0.4602 - acc: 0.7803


0.7424242496490479
2968 2968
446767104 2968
Epoch 1/2
 72/149 [=============>................] - ETA: 1:49 - loss: 0.4339 - acc: 0.7951

KeyboardInterrupt: 